<a href="https://colab.research.google.com/github/AtacZy/Projects/blob/main/%D0%9A%D0%B5%D0%B9%D1%81_1_%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_csv_%D1%84%D0%B0%D0%B9%D0%BB%D0%BE%D0%B2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обработка чеков о продажах в формате 'csv'.

##**Цель:**
 Написать функцию procces_files, которая принимает на вход два пути к папкам. Из первой папки необходимо выбрать все "чеки" (файлы по шаблону из условия), а во второую папку сохранить один объединённый чек (отсортированный по дате, потом по продукту) под названием combined_data.csv.



---


##**Автор**: AtacZy <br>
##**Дата**: 10.02 <br>

##**Данные для работы ниже**:

In [ ]:
!wget https://github.com/vs8th/reports/archive/main.zip

import zipfile

with zipfile.ZipFile("main.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

!rm main.zip

##**Как должны выглядеть данные на выходе**:

In [ ]:
import pandas as pd

df = pd.read_csv('reports-main/2023-02-17-05-38-2.csv', sep=";")
df

,date,product,store,cost
0,2023-02-17,product_0,store_2,10
1,2023-02-17,product_1,store_2,20
2,2023-02-17,product_2,store_2,30
3,2023-02-17,product_3,store_2,40
4,2023-02-17,product_4,store_2,50


##**Логика**:
1. Нужно отобрать файлы формата csv в нужной папке;
2. Потом отфильтровать по названию файла с помощью регулярного выражения;
3. Потом каждый файл прочитать и объединить в один список, словарь;
4. Отсортировать по названию и дате;
5. Записать данные в один csv-файл в папку назначения;
6. Вызвать функцию;

In [ ]:
import glob
import os
import re
import csv

def process_files(src_folder, dest_folder):
# 1. Получаем список всех файлов формата csv в нужной папке
    all_files = glob.glob(src_folder + '/*.csv')
    file_list = []
# 2. Отфильтровываем те, что не подходят по шаблону
    for file in all_files:
        if re.match(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d+.csv', os.path.basename(file)):
            file_list.append(file)

# 3. Объединяем содержимое всех csv-файлов в единый список словарей
    combined_data = []
    for file in file_list:
        with open(file, 'r') as f:
# 3.1 Предположим, что первая строка csv-файла - это заголовок
            header = f.readline().strip().split(';')
            for line in f:
                data = line.strip().split(';')
                combined_data.append(dict(zip(header, data)))

# 4. Сортируем объединенные данные по заданному параметру
    combined_data = sorted(combined_data, key=lambda x: x['date'])

# 5. Записываем объединенные данные в один csv-файл в папку назначения
    dest_file = os.path.join(dest_folder, 'combined_data.csv')
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    with open(dest_file, 'w', encoding='utf-8', newline='') as f:
      if combined_data:
        writer = csv.DictWriter(f, fieldnames=combined_data[0].keys(), extrasaction='ignore')
        writer.writeheader()
        writer.writerows(combined_data)

# 6. Указываем все данные - и вызываем функцию
src_folder = 'reports-main'
dest_folder = 'folder2'
process_files(src_folder, dest_folder)

##Для проверки выполнить два кода ниже:

In [ ]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv

import pandas as pd

user_answer = pd.read_csv(f'{dest_folder}/combined_data.csv')
correct_answer = pd.read_csv('data.csv')

In [ ]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')